<a id='start'></a>

In [3]:
# Attempt to implement Tienet in a day
import torchvision
import torch
from torch import nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.nn.utils.rnn import pack_padded_sequence
from torchvision import transforms
import os, time
from tqdm import tqdm_notebook as tqdm
import numpy as np 
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import re
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline 
from sklearn.metrics.ranking import roc_auc_score

from DatasetGenerator import *
from ChexnetTrainer import masked_BCE, DataLoader, TieNetLoss

[Jump to training if functions have already been loaded](#train)<br><br>

[Jump to training if functions haven't been loaded](#train2)<br>


## We first make the train and valtest .txt files 

In [4]:
# Need to generate the txt file for train and validation
txt_file_path = './dataset/train_tienet.txt'
labels_paths = ['../data_4/labels_20k_4.npy',
           '../data_5/labels_5.npy', 
           '../data_6/labels_6.npy',
               '../data_2/labels_2.npy',
               '../data_7/labels_7.npy',
               '../data_8/labels_8.npy']
img_dirs = ['../data_4/data/tiffs_20k_4',#]#, 
           '../data_5/data/tiffs_5', 
           '../data_6/data/tiffs_6',
           '../data_2/data/tiffs_2',
           '../data_7/data/tiffs_7',
           '../data_8/data/tiffs_8']
report_paths = ['../data_4/reports_20k_4',#]#, 
           '../data_5/reports_5', 
           '../data_6/reports_6',
           '../data_2/reports_2',
           '../data_7/reports_7',
           '../data_8/reports_8']

########################

labels = []
for labels_path in labels_paths:
    labels.append(np.load(labels_path, encoding='bytes'))
total_labels = np.concatenate(tuple(labels), axis=0)


names_list = []
for img_dir in img_dirs:
    temp_names_list = [' ']*len(os.listdir(img_dir))
    temp_dir_header = img_dir[3:]
    print(temp_dir_header)
    for fname in os.listdir(img_dir):
        # Get actual index 
        index = int(fname.split('_')[0])
        temp_names_list[index] = os.path.join(temp_dir_header, fname)
    names_list += temp_names_list
    
reports_list = []
for report_dir in report_paths:
    temp_names_list = [' ']*len(os.listdir(report_dir))
    temp_dir_header = report_dir[3:]
    print(temp_dir_header)
    for fname in os.listdir(report_dir):
        # Get actual index 
        index = int(fname.split('_')[0])
        temp_names_list[index] = os.path.join(temp_dir_header, fname)
    reports_list += temp_names_list

d = {'filename':names_list}
dtext = {'reportname':reports_list}
df1 = pd.DataFrame(data=d)
df1 = pd.concat((df1, pd.DataFrame(data=dtext)),axis=1)
df2 = pd.DataFrame(data=total_labels)
df2 = df2.astype('int')

df = pd.concat((df1,df2),axis=1)

df.to_csv(txt_file_path, sep=' ', index = False, header=False)

data_4/data/tiffs_20k_4
data_5/data/tiffs_5
data_6/data/tiffs_6
data_2/data/tiffs_2
data_7/data/tiffs_7
data_8/data/tiffs_8
data_4/reports_20k_4
data_5/reports_5
data_6/reports_6
data_2/reports_2
data_7/reports_7
data_8/reports_8


In [5]:
# Need to generate the txt file for train and validation
txt_file_path = './dataset/valtest_tienet.txt'
labels_paths = ['../data_3/labels_20k_3.npy']
img_dirs = ['../data_3/data/tiffs_3']
report_paths = ['../data_3/reports_20k_3']#, 
#            '../data_5/reports_5', 
#            '../data_6/reports_6',
#            '../data_2/reports_2',
#            '../data_7/reports_7',
#            '../data_8/reports_8']

########################

labels = []
for labels_path in labels_paths:
    labels.append(np.load(labels_path, encoding='bytes'))
total_labels = np.concatenate(tuple(labels), axis=0)


names_list = []
for img_dir in img_dirs:
    temp_names_list = [' ']*len(os.listdir(img_dir))
    temp_dir_header = img_dir[3:]
    print(temp_dir_header)
    for fname in os.listdir(img_dir):
        # Get actual index 
        index = int(fname.split('_')[0])
        temp_names_list[index] = os.path.join(temp_dir_header, fname)
    names_list += temp_names_list
    
reports_list = []
for report_dir in report_paths:
    temp_names_list = [' ']*len(os.listdir(report_dir))
    temp_dir_header = report_dir[3:]
    print(temp_dir_header)
    for fname in os.listdir(report_dir):
        # Get actual index 
        index = int(fname.split('_')[0])
        temp_names_list[index] = os.path.join(temp_dir_header, fname)
    reports_list += temp_names_list

d = {'filename':names_list}
dtext = {'reportname':reports_list}
df1 = pd.DataFrame(data=d)
df1 = pd.concat((df1, pd.DataFrame(data=dtext)),axis=1)
df2 = pd.DataFrame(data=total_labels)
df2 = df2.astype('int')

df = pd.concat((df1,df2),axis=1)

df.to_csv(txt_file_path, sep=' ', index = False, header=False)

data_3/data/tiffs_3
data_3/reports_20k_3


In [7]:
# Convert valtest into val and test
val_txt_file_path = './dataset/val_tienet.txt'
test_txt_file_path = './dataset/test_tienet.txt'
index_to_split = 8712

df = pd.read_csv(txt_file_path, sep=' ', header=None)
df_val = df[:index_to_split]
df_test = df[index_to_split:]
df_val.to_csv(val_txt_file_path, sep=' ', index=False, header=False)
df_test.to_csv(test_txt_file_path, sep=' ', index=False, header=False)

## 2) Text processing from the reports 

In [22]:
# Preprocess the reports 
report_paths = ['../data_4/reports_20k_4',#]#, 
           '../data_5/reports_5', 
           '../data_6/reports_6',
           '../data_2/reports_2',
           '../data_7/reports_7',
           '../data_8/reports_8'] # need to make sure this is the same order as the rpeorts_path above 

In [23]:
def bioclean(t):
    return re.sub('[.,?;*!%^&_+():-\[\]{}]', '', t.replace('"', '').replace('/', '').replace('\\', '').replace("'",'').strip().lower()).split()

In [24]:
# Preprocess the reports
processed_reports = [] 
for report_dir in report_paths:
    filenames = os.listdir(report_dir)
    for filename in filenames:
        with open(os.path.join(report_dir, filename),'r') as f:
            g = f.read().replace('\n', '')
        f.close()
        tokens = bioclean(g)
        processed_reports.append(tokens) 
    

In [28]:
# For training word2vec model

pretrained_word2vec_path = 'pubmed2018_w2v_200D/pubmed2018_w2v_200D.bin'
model_path = 'data_456278_pubmed_embeddings'
weights_path = 'data_456278_weights.npy'

################

word_vectors = KeyedVectors.load_word2vec_format(pretrained_word2vec_path, binary=True)

model = Word2Vec(size=200, min_count=2, workers=16)
model.build_vocab(processed_reports)
total_examples = model.corpus_count

model.build_vocab([list(word_vectors.vocab.keys())], update=True)

model.intersect_word2vec_format(pretrained_word2vec_path, binary=True, lockf=1.0)

model.train(processed_reports, total_examples=total_examples, epochs=model.epochs)

# save the 
model.wv.save(model_path)

In [29]:
# Make a weight matrix 
model = KeyedVectors.load(model_path)
weights_numpy = np.concatenate((np.zeros((1,*model.vectors.shape[1:])),model.vectors),axis=0) # add 0's for uncommon words 
weights = torch.FloatTensor(weights_numpy) # formerly syn0, which is soon deprecated

# add start and end tokens
eps = 1e-3 # the standard deviation of the small random vector we choose for start and end tokens 
start_token = torch.FloatTensor(np.random.randn(1,weights.shape[1])*eps)
end_token = torch.FloatTensor(np.random.randn(1,weights.shape[1])*eps)
weights = torch.cat((weights,start_token,end_token))

In [30]:
# Save the weights matrix 
np.save(weights_path,weights.cpu().numpy())

In [45]:
print(weights.shape, len(model.vocab))

torch.Size([15242, 200]) 15239


In [328]:
# # Make thew eight matrix 
# embed = nn.Embedding(weights.shape[0],weights.shape[1],padding_idx=0)
# embed2 = embed.from_pretrained(weights,freeze=False)

# emb_layer, _, _ = create_emb_layer(weights, non_trainable=False, padding_idx=0)

In [9]:
# # The pipeline for reading a report and indexing a report
# tokens = tokenize_report(os.path.join(report_dir, filenames[0]))
# report = index_report(tokens, model, weights.shape[0], include_start_end=True)

## Make the TieNet model

<a id='train2'></a>

In [35]:
# def create_emb_layer(weights_matrix, non_trainable=False, padding_idx=0):
#     num_embeddings, embedding_dim = weights_matrix.size()
#     emb_layer = nn.Embedding(num_embeddings, embedding_dim, padding_idx=padding_idx)
#     emb_layer.load_state_dict({'weight': weights_matrix})
#     if non_trainable:
#         emb_layer.weight.requires_grad = False

#     return emb_layer


# class ResNet50(nn.Module):
#     def __init__(self, classCount, isTrained, trainable=False):
    
#         super(ResNet50, self).__init__()
        
#         self.resnet50 = torchvision.models.resnet50(pretrained=isTrained)
#         # Take out the global average pooling and fully connected layers 
#         self.features = nn.Sequential(*list(self.resnet50.children())[:-2])
#         for child in self.features.children():
#             for param in child.parameters():
#                 if trainable:
#                     param.requires_grad = True
#                 else:
#                     param.requires_grad = False 
#     def forward(self, x):
#         x = self.features(x)
#         return x


                
# class ImageEncoder(nn.Module):
#     # Assumes use of 224*224 images 
#     def __init__(self, transition_dim, lstm_hidden_dim):
#         super(ImageEncoder, self).__init__()
#         #
#         self.cnn = ResNet50(14, True, trainable=False)
#         self.conv1 = nn.Conv2d(2048, transition_dim, 1)  
#         #
        
#     def forward(self, x):
#         x = self.cnn(x) # Initial feature extraction 
#         x = self.conv1(x) # transition conv layer
#         x = F.relu(x) 
#         # need to get the activity a0 to elementwise multiply with X 
#         return x

# class AttentionLSTMDecoder(nn.Module):
#     # LSTM decoder component of TieNet 
#     def __init__(self, transition_dim, lstm_hidden_dim, embedding_dim, output_dim, weights):
#         self.embedding_dim = embedding_dim
#         self.hidden_dim = lstm_hidden_dim
#         self.output_dim = output_dim
#         self.encode_output_dim = encode_output_dim
#         self.weights = weights
        
#         self.transition = nn.Linear(transition_dim*7*7, lstm_hidden_dim)
#         self.embed = create_embed_layer(weights, non_trainable=False, padding_idx=0)
        
#     def forward(self, word, hidden, encoded):
#         hidden = self.transition(encoded.view(encoded.size(0),-1))
#         word_embedded = self.embed(word)
    

class Encoder(nn.Module):
    """
    Encoder.
    """

    def __init__(self, transition_dim=1024):
        super(Encoder, self).__init__()

        resnet = torchvision.models.resnet50(pretrained=True)  # pretrained ImageNet ResNet-101

        # Remove linear and pool layers (since we're not doing classification)
        modules = list(resnet.children())[:-2]
        self.resnet = nn.Sequential(*modules)
        self.fine_tune(fine_tune=False) # don't make any layers of ResNet trainable 
        
        ## Resize image to fixed size to allow input images of variable size
        #self.adaptive_pool = nn.AdaptiveAvgPool2d((encoded_image_size, encoded_image_size))
        
        self.transition_conv = nn.Conv2d(2048, transition_dim, 1) # output shape (1024, 7, 7)
#         self.transition_conv = nn.ConvTranspose2d(2048, transition_dim, 10) # output shape (1024, 16, 16) as in the paper
        

    def forward(self, images):
        """
        Forward propagation.
        :param images: images, a tensor of dimensions (batch_size, 3, image_size, image_size)
        :return: encoded images
        """
        out = self.resnet(images)  # (batch_size, 2048, image_size/32, image_size/32)
#         out = self.adaptive_pool(out)  # (batch_size, 2048, encoded_image_size, encoded_image_size)
#         out = out.permute(0, 2, 3, 1)  # (batch_size, encoded_image_size, encoded_image_size, 2048)

        out = self.transition_conv(out) 
        out = F.relu(out) 
        out = out.permute(0,2,3,1) # (batch_size, encoded_image_size, encoded_image_size, 1024) 
        
        return out

    def fine_tune(self, fine_tune=False):
        """
        Allow or prevent the computation of gradients for convolutional blocks 2 through 4 of the encoder.
        :param fine_tune: Allow?
        """
        for p in self.resnet.parameters():
            p.requires_grad = False
        # If fine-tuning, only fine-tune convolutional blocks 2 through 4
        for c in list(self.resnet.children())[5:]:
            for p in c.parameters():
                p.requires_grad = fine_tune


    
class Attention(nn.Module):
    """
    Attention Network.
    """

    def __init__(self, encoder_dim, decoder_dim, attention_dim):
        """
        :param encoder_dim: feature size of encoded images
        :param decoder_dim: size of decoder's RNN
        :param attention_dim: size of the attention network
        """
        super(Attention, self).__init__()
        self.encoder_att = nn.Linear(encoder_dim, attention_dim)  # linear layer to transform encoded image
        self.decoder_att = nn.Linear(decoder_dim, attention_dim)  # linear layer to transform decoder's output
        self.full_att = nn.Linear(attention_dim, 1)  # linear layer to calculate values to be softmax-ed
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)  # softmax layer to calculate weights

    def forward(self, encoder_out, decoder_hidden):
        """
        Forward propagation.
        :param encoder_out: encoded images, a tensor of dimension (batch_size, num_pixels, encoder_dim)
        :param decoder_hidden: previous decoder output, a tensor of dimension (batch_size, decoder_dim)
        :return: attention weighted encoding, weights
        """
        att1 = self.encoder_att(encoder_out)  # (batch_size, num_pixels, attention_dim)
        att2 = self.decoder_att(decoder_hidden)  # (batch_size, attention_dim)
        att = self.full_att(self.relu(att1 + att2.unsqueeze(1))).squeeze(2)  # (batch_size, num_pixels)
        alpha = self.softmax(att)  # (batch_size, num_pixels)
        attention_weighted_encoding = (encoder_out * alpha.unsqueeze(2)).sum(dim=1)  # (batch_size, num_pixels, encoder_dim)

        return attention_weighted_encoding, alpha
    
    
    

        

In [38]:
class DecoderWithAttention(nn.Module):
    """
    Decoder. Also includes Saliency-weighted global average pooling and attention encoded text embedding.
    """

    def __init__(self, attention_dim, embed_dim, decoder_dim, vocab_size, 
                 num_classes=14, aete_num_global_attentions=5, aete_dimensionality=2000, 
                 encoder_dim=1024, dropout=0.5):
        """
        :param attention_dim: size of attention network
        :param embed_dim: embedding size
        :param decoder_dim: size of decoder's RNN 
        :param vocab_size: size of vocabulary
        :param encoder_dim: feature size of encoded images
        :param dropout: dropout
        :num_classes: the number of classes for multilabel training (default is 14 for Chest X rays)
        :num_global_attentions: r parameter (TieNet paper uses 5)
        :dimensionality: s parameter (TieNet paper uses 2000)
        """
        super(DecoderWithAttention, self).__init__()

        self.encoder_dim = encoder_dim
        self.attention_dim = attention_dim
        self.embed_dim = embed_dim
        self.decoder_dim = decoder_dim
        self.vocab_size = vocab_size
        self.dropout = dropout
        
        
        self.aete = AETE(decoder_dim, aete_num_global_attentions, aete_dimensionality) # attention encoded text embedding
        self.swgap = SaliencyGlobalAvgPool(aete_num_global_attentions) # saliency-weighted global avg pool
        #self.poolt = nn.MaxPool1d(aete_num_global_attentions)
        self.attention = Attention(encoder_dim, decoder_dim, attention_dim)  # attention network

        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)  # embedding layer
        self.dropout = nn.Dropout(p=self.dropout)
        self.decode_step = nn.LSTMCell(embed_dim + encoder_dim, decoder_dim, bias=True)  # decoding LSTMCell
        self.init_h = nn.Linear(encoder_dim, decoder_dim)  # linear layer to find initial hidden state of LSTMCell
        self.init_c = nn.Linear(encoder_dim, decoder_dim)  # linear layer to find initial cell state of LSTMCell
        self.f_beta = nn.Linear(decoder_dim, encoder_dim)  # linear layer to create a sigmoid-activated gate
        self.sigmoid = nn.Sigmoid()
        self.fc = nn.Linear(decoder_dim, vocab_size)  # linear layer to find scores over vocabulary
        self.fc_final = nn.Linear(decoder_dim+encoder_dim, num_classes)
        self.init_weights()  # initialize some layers with the uniform distribution

    def init_weights(self):
        """
        Initializes some parameters with values from the uniform distribution, for easier convergence.
        """
        #self.embedding.weight.data.uniform_(-0.1, 0.1)
        self.fc.bias.data.fill_(0)
        self.fc.weight.data.uniform_(-0.1, 0.1)

    def load_pretrained_embeddings(self, embeddings):
        """
        Loads embedding layer with pre-trained embeddings.
        :param embeddings: pre-trained embeddings
        """
        self.embedding.weight = nn.Parameter(embeddings)
        #self.embedding.load_state_dict({'weight': embeddings})

    def fine_tune_embeddings(self, fine_tune=True):
        """
        Allow fine-tuning of embedding layer? (Only makes sense to not-allow if using pre-trained embeddings).
        :param fine_tune: Allow?
        """
        for p in self.embedding.parameters():
            p.requires_grad = fine_tune

    def init_hidden_state(self, encoder_out):
        """
        Creates the initial hidden and cell states for the decoder's LSTM based on the encoded images.
        :param encoder_out: encoded images, a tensor of dimension (batch_size, num_pixels, encoder_dim)
        :return: hidden state, cell state
        """
        mean_encoder_out = encoder_out.mean(dim=1)
        h = self.init_h(mean_encoder_out)  # (batch_size, decoder_dim)
        c = self.init_c(mean_encoder_out)
        return h, c

    def forward(self, encoder_out, encoded_captions, caption_lengths):
        """
        Forward propagation.
        :param encoder_out: encoded images, a tensor of dimension (batch_size, enc_image_size, enc_image_size, encoder_dim)
        :param encoded_captions: encoded captions, a tensor of dimension (batch_size, max_caption_length)
        :param caption_lengths: caption lengths, a tensor of dimension (batch_size, 1)
        :return: scores for vocabulary, decode lengths, sort indices, classification output
        """

        batch_size = encoder_out.size(0)
        encoder_dim = encoder_out.size(-1)
        vocab_size = self.vocab_size

        # Flatten image
        encoder_out = encoder_out.view(batch_size, -1, encoder_dim)  # (batch_size, num_pixels, encoder_dim)
        num_pixels = encoder_out.size(1)

        # Sort input data by decreasing lengths; why? apparent below
        caption_lengths, sort_ind = caption_lengths.sort(dim=0, descending=True)
        encoder_out = encoder_out[sort_ind]
        encoded_captions = encoded_captions[sort_ind]

        # Embedding
        embeddings = self.embedding(encoded_captions)  # (batch_size, max_caption_length, embed_dim)

        # Initialize LSTM state
        h, c = self.init_hidden_state(encoder_out)  # (batch_size, decoder_dim)

        # We won't decode at the <end> position, since we've finished generating as soon as we generate <end>
        # So, decoding lengths are actual lengths - 1
        decode_lengths = (caption_lengths - 1).tolist()

        # Create tensors to hold word predicion scores and alphas
        predictions = torch.zeros(batch_size, max(decode_lengths), vocab_size).to(device)
        alphas = torch.zeros(batch_size, max(decode_lengths), num_pixels).to(device)
        hiddens = torch.zeros(batch_size, max(decode_lengths), h.size(-1)).to(device)

        # At each time-step, decode by
        # attention-weighing the encoder's output based on the decoder's previous hidden state output
        # then generate a new word in the decoder with the previous word and the attention weighted encoding
        for t in range(max(decode_lengths)):
            batch_size_t = sum([l > t for l in decode_lengths])
            attention_weighted_encoding, alpha = self.attention(encoder_out[:batch_size_t],
                                                                h[:batch_size_t])
            gate = self.sigmoid(self.f_beta(h[:batch_size_t]))  # gating scalar, (batch_size_t, encoder_dim)
            attention_weighted_encoding = gate * attention_weighted_encoding
            h, c = self.decode_step(
                torch.cat([embeddings[:batch_size_t, t, :], attention_weighted_encoding], dim=1),
                (h[:batch_size_t], c[:batch_size_t]))  # (batch_size_t, decoder_dim)
            preds = self.fc(self.dropout(h))  # (batch_size_t, vocab_size)
            predictions[:batch_size_t, t, :] = preds
            alphas[:batch_size_t, t, :] = alpha
            hiddens[:batch_size_t, t, :] = h
            
        # Attention encoded text embedding and saliency-weighted global average pooling 
        x_aete, g = self.aete(hiddens, decode_lengths)
        x_swgap = self.swgap(g, alphas, encoder_out)
    
    
        # concatenate these two 
        x = torch.cat([x_aete, x_swgap], dim=-1)
        output = self.fc_final(x)
        
        return predictions, decode_lengths, sort_ind, output, encoded_captions 

In [39]:
# Now make the Attention encoded text embedding part of the network

class AETE(nn.Module):
    """
    Attention Encoded Text Embedding.
    """
    
    def __init__(self, decoder_dim, num_global_attentions, dimensionality):
        # num_global_attentions refers to "r" parameter in paper, set to 5
        # dimensionality refers to "s" parameter in paper, set to 2000
        super(AETE, self).__init__()
        self.Ws2 = nn.Linear(dimensionality, num_global_attentions)
        self.Ws1 = nn.Linear(decoder_dim, dimensionality)
        self.tanh = nn.Tanh() 
        self.softmax = nn.Softmax(dim=1) 
        self.pool = nn.MaxPool1d(num_global_attentions)
        self.num_global_attentions = num_global_attentions
    
    def forward(self, hidden_states, decode_lengths):
        # hidden_states contains the whole sequence (in time) of hidden states (batch_size, caption_lengths, decoder_dim)
        batch_size = hidden_states.size(0)
        decoder_dim = hidden_states.size(-1)
        
        ms = torch.zeros(batch_size, decoder_dim).to(device)
        gs = torch.zeros(batch_size, max(decode_lengths), self.num_global_attentions).to(device)
        for ind, t in enumerate(decode_lengths): 
            hidden_state = hidden_states[ind]
            hidden_state = hidden_state[:t]
            g = self.Ws1(hidden_state) # shape (caption_lengths, dimensionality)
            g = self.tanh(g) 
            g = self.Ws2(g) # (caption_lengths, num_global_attentions)
            g = self.softmax(g)
            m = torch.matmul(hidden_state.t(), g) # (decoder_dim, num_global_attentions)
            #print(m.shape)
            m = self.pool(m.unsqueeze(0)).squeeze() # (decoder_dim,)
            ms[ind] = m # this is the X_aete that we use for joint learning 
            gs[ind,:t] = g
            
        return ms, gs
    
# We make the saliency weighted global average pooling
class SaliencyGlobalAvgPool(nn.Module):
    """
    Saliency Weighted Global Average Pooling
    """
    def __init__(self, num_global_attentions):
        super(SaliencyGlobalAvgPool, self).__init__()
        self.pool = nn.MaxPool1d(num_global_attentions)
        
    def forward(self, g, alpha, encoder_output):
        g = self.pool(g) # g input is (batch_size, decoder_length, num_global_attentions) --. (batch_size,decoder_length,1)
        # Multiply these vectors by the image attention 
        # size alpha = (batch_size, decoder_length, num_pixels)
        aws = g*alpha # (batch_size, decoder_length, num_pixels)
        #aws = torch.mul(g, alpha)
        aws = aws.sum(dim=1) # (batch_size, num_pixels)
        # Now multiply by the encoded image 
        x_swgap = aws.unsqueeze(2) * encoder_output # (batch_size, num_pixels, encoder_dim)
        x_swgap = x_swgap.sum(dim=1) # (batch_size, encoder_dim)
        return x_swgap 

In [40]:
def train(train_loader, encoder, decoder, criterion, encoder_optimizer, decoder_optimizer, epoch):
    """
    Performs one epoch's training.
    :param train_loader: DataLoader for training data
    :param encoder: encoder model
    :param decoder: decoder model
    :param criterion: loss layer
    :param encoder_optimizer: optimizer to update encoder's weights (if fine-tuning)
    :param decoder_optimizer: optimizer to update decoder's weights
    :param epoch: epoch number
    """

    decoder.train()  # train mode (dropout and batchnorm is used)
    encoder.train()

    batch_time = AverageMeter()  # forward prop. + back prop. time
    data_time = AverageMeter()  # data loading time
    losses = AverageMeter()  # loss (per word decoded)
    auroc_avg = AverageMeter() # auroc per sample 

    start = time.time()

    # Batches
    for i, (imgs, text, textlens, label) in tqdm(enumerate(train_loader)):
        data_time.update(time.time() - start)

        # Move to GPU, if available
        imgs = imgs.cuda(async=True)
        text = text.cuda(async=True)
        textlens = textlens.cuda(async=True)
        label = label.cuda(async=True)

        # Forward prop.
        imgs = encoder(imgs) # (batch_size, img_dim, img_dim, encoder_dim)
        
        # scores = word predictions 
        word_predictions, decode_lengths, sort_ind, output, text_sorted= decoder(imgs, text, textlens)
        label = label[sort_ind] # since they were re-sorted
        
        # Since we decoded starting with <start>, the targets are all words after <start>, up to <end>
        targets = text_sorted[:, 1:] # this is the target words that we are trying to predict 
        # Remove timesteps that we didn't decode at, or are pads
        # pack_padded_sequence is an easy trick to do this
        scores, _ = pack_padded_sequence(word_predictions, decode_lengths, batch_first=True)
        targets, _ = pack_padded_sequence(targets, decode_lengths, batch_first=True)
        
        # Calculate loss
        loss = criterion(output, label, scores, targets)

        # Back prop.
        decoder_optimizer.zero_grad()
        if encoder_optimizer is not None:
            encoder_optimizer.zero_grad()
        loss.backward()

        # Clip gradients
        if grad_clip is not None:
            clip_gradient(decoder_optimizer, grad_clip)
            if encoder_optimizer is not None:
                clip_gradient(encoder_optimizer, grad_clip)

        # Update weights
        decoder_optimizer.step()
        if encoder_optimizer is not None:
            encoder_optimizer.step()

        # Keep track of metrics
#         auroc = computeAUROC(label, output, 14)[-1] # get the overall AUROC of batch 
        losses.update(loss.item(), len(decode_lengths))
        batch_time.update(time.time() - start)
#         auroc_avg.update(auroc, len(decode_lengths))

        start = time.time()

        # Print status
        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data Load Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(epoch, i, len(train_loader),
                                                                          batch_time=batch_time,
                                                                          data_time=data_time, loss=losses))
                                                                          #auroc=auroc, auroc_avg=auroc_avg))

In [41]:
def validate(val_loader, encoder, decoder, criterion):
    """
    Performs one epoch's validation.
    :param val_loader: DataLoader for validation data.
    :param encoder: encoder model
    :param decoder: decoder model
    :param criterion: loss layer
    :return: roc_auc score 
    """
    decoder.eval()  # eval mode (no dropout or batchnorm)
    if encoder is not None:
        encoder.eval()

#     batch_time = AverageMeter()
    losses = AverageMeter()
#     aucs = AverageMeter()

#     start = time.time()


    # explicitly disable gradient calculation to avoid CUDA memory error
    # solves the issue #57
    
    labels = torch.FloatTensor().cuda()
    outputs = torch.FloatTensor().cuda()
    with torch.no_grad():
        # Batches
        for i, (imgs, text, textlens, label) in enumerate(val_loader):

            # Move to device, if available
            imgs = imgs.to(device)
            text = text.to(device)
            textlens = textlens.to(device)
            label = label.to(device)

            # Forward prop.
            if encoder is not None:
                imgs = encoder(imgs)
            word_predictions, decode_lengths, sort_ind, output, text_sorted = decoder(imgs, text, textlens)
            label = label[sort_ind]

            # Calculate loss
            # Since we decoded starting with <start>, the targets are all words after <start>, up to <end>
            targets = text_sorted[:, 1:] # this is the target words that we are trying to predict 
            # Remove timesteps that we didn't decode at, or are pads
            # pack_padded_sequence is an easy trick to do this
            scores, _ = pack_padded_sequence(word_predictions, decode_lengths, batch_first=True)
            targets, _ = pack_padded_sequence(targets, decode_lengths, batch_first=True)
            loss = criterion(output, label, scores, targets)
            outputs = torch.cat((outputs, output), 0)
            labels = torch.cat((labels, label), 0)

            # Keep track of metrics
            losses.update(loss.item(), len(decode_lengths))
            
    aucs = computeAUROC(labels, outputs, 14)
    print("Validation AUC:", aucs[-1])
    print("Specific AUCs:", aucs[:-1])
    print("Validation Loss:", losses.avg)
    return aucs

## Utils

In [42]:
def clip_gradient(optimizer, grad_clip):
    """
    Clips gradients computed during backpropagation to avoid explosion of gradients.
    :param optimizer: optimizer with the gradients to be clipped
    :param grad_clip: clip value
    """
    for group in optimizer.param_groups:
        for param in group['params']:
            if param.grad is not None:
                param.grad.data.clamp_(-grad_clip, grad_clip)
                
def adjust_learning_rate(optimizer, shrink_factor):
    """
    Shrinks learning rate by a specified factor.
    :param optimizer: optimizer whose learning rate must be shrunk.
    :param shrink_factor: factor in interval (0, 1) to multiply learning rate with.
    """

    print("\nDECAYING learning rate.")
    for param_group in optimizer.param_groups:
        param_group['lr'] = param_group['lr'] * shrink_factor
    print("The new learning rate is %f\n" % (optimizer.param_groups[0]['lr'],))
    
    
def save_checkpoint(data_name, epoch, epochs_since_improvement, encoder, decoder, encoder_optimizer, decoder_optimizer,
                    overall_auroc, class_aurocs, is_best):
    """
    Saves model checkpoint.
    :param data_name: base name of processed dataset
    :param epoch: epoch number
    :param epochs_since_improvement: number of epochs since last improvement in BLEU-4 score
    :param encoder: encoder model
    :param decoder: decoder model
    :param encoder_optimizer: optimizer to update encoder's weights, if fine-tuning
    :param decoder_optimizer: optimizer to update decoder's weights
    :param bleu4: validation auroc
    :param is_best: is this checkpoint the best so far?
    """
    state = {'epoch': epoch,
             'epochs_since_improvement': epochs_since_improvement,
             'overall_auroc': overall_auroc,
             'class_aurocs': class_aurocs, 
             'encoder': encoder.state_dict(),
             'decoder': decoder.state_dict(),
             'encoder_optimizer': encoder_optimizer.state_dict(),
             'decoder_optimizer': decoder_optimizer.state_dict()}
    filename = 'checkpoint_' + data_name + '.pth.tar'
    torch.save(state, 'models/'+filename)
    # If this checkpoint is the best so far, store a copy so it doesn't get overwritten by a worse checkpoint
    if is_best:
        torch.save(state, 'models/BEST_' + filename)
        
class AverageMeter(object):
    """
    Keeps track of most recent, average, sum, and count of a metric.
    """

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
        
def computeAUROC (dataGT, dataPRED, classCount):
        
        outAUROC = []
        
        datanpGT = dataGT.cpu().detach().numpy()
        datanpPRED = dataPRED.cpu().detach().numpy()


        total_num_samples = [] 
        for i in range(classCount):
            datanpGT_temp = datanpGT[:,i] 
            datanpGT_nonneg = datanpGT_temp[datanpGT_temp >= 0]
            datanpPRED_nonneg = datanpPRED[:,i]
            datanpPRED_nonneg = datanpPRED_nonneg[datanpGT_temp >= 0]
            num_samples = len(datanpGT_nonneg)
            outAUROC.append(roc_auc_score(datanpGT_nonneg, datanpPRED_nonneg))
            total_num_samples.append(num_samples)
            #print(outAUROC[i], num_samples)

        overallAUROC = np.sum(np.array(outAUROC)*np.array(total_num_samples))/np.sum(total_num_samples)

        outAUROC.append(overallAUROC)
        
        return outAUROC

## Actually train 
<a id='train'></a>

[Click here to go to evaluation code](#eval)<br>

In [43]:
# First, for the loss function we want to weight the classes so we first compute those parameters
# First compute the class weights for the training data set 
pathFileTrain = 'dataset/train_tienet.txt'
actual_labels = pd.read_csv(pathFileTrain,delimiter=' ',header=None)
#actual_labels.head()
act = actual_labels[actual_labels.columns[2:]]
actarray = np.array(act)

lambda_m = np.array([(len(arr[arr>=0])-arr[arr>=0].sum())/len(arr[arr>=0]) for arr in actarray.transpose()]) 
beta_N = actarray[actarray >= 0].sum()/len(actarray[actarray >= 0])
beta_P = [(1-beta_N)/beta_N]

print(lambda_m, beta_N, beta_P)

[0.67177793 0.9671413  0.79564277 0.76717925 0.97223951 0.87407297
 0.95112196 0.91820742 0.78678206 0.95289435 0.75602778 0.99179818
 0.98021823 0.7062537 ] 0.13653332124909798 [6.324219398248958]


In [ ]:
# Code for training

# Parameters
# Data parameters
data_name = 'tienet20k_dualloss_alldata'# for checkpoint 
train_report_dirs = ['../data_4/reports_20k_4',#]#, 
           '../data_5/reports_5', 
           '../data_6/reports_6',
           '../data_2/reports_2',
           '../data_7/reports_7',
           '../data_8/reports_8']
val_report_dir = '../data_3/reports_20k_3'
pathDirDataTrain = '..'
pathDirDataVal = '..' #None # if none, don't use validation 
pathFileTrain = 'dataset/train_tienet.txt'
pathFileVal = 'dataset/val_tienet.txt'
embedding_weight_path = 'data4_weights.npy'
embeddingModelFile = 'data_4_pubmed_embeddings'

# Preprocessing
transCrop = 224 # size of the final image
longest_train = 539 # if we know the longest sequence (remember to add 2 for the start/end tokens)
longest_val = 348

# Model parameters
emb_dim = 200  # dimension of word embeddings
vocab_size = 15242 # number of words in vocabulary, including <start>, <end>, and pad index/unknown words (0)
encoder_dim = 1024 # dimension of the encoded images 
attention_dim = 1024  # dimension of attention linear layers, originally 350
decoder_dim = 256 # dimension of decoder RNN
aete_num_global_attentions = 5 # for attention encoded text embedding
aete_dimensionality = 2000 # for attention encoded text embedding
num_classes = 14 # number of classes for classification
dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # sets device for model and PyTorch tensors
cudnn.benchmark = False  # set to true only if inputs to model are fixed size; otherwise lot of computational overhead

# Training parameters
epochs = 100  # number of epochs to train for (if early stopping is not triggered)
patience = 10 # number of epochs to wait for improvement before stopping 
batch_size = 32
alpha = 0.3 # how much to wait the classification loss vs. the RNN loss 
num_workers = 12  # for data-loading
encoder_lr = 1e-3  # learning rate for encoder if fine-tuning
decoder_lr = 1e-3  # learning rate for decoder
grad_clip = 5.  # clip gradients at an absolute value of
print_freq = 100  # print training/validation stats every __ batches
fine_tune_encoder = False  # fine-tune encoder? No 
checkpoint = None# 'models/checkpoint_'+data_name+'.pth.tar'  # path to checkpoint, None if none

###############################################################################
## Actual model



if checkpoint is None:
    decoder = DecoderWithAttention(attention_dim=attention_dim,
                                   embed_dim=emb_dim,
                                   decoder_dim=decoder_dim,
                                   vocab_size=vocab_size,
                                   num_classes=num_classes, 
                                   aete_num_global_attentions=aete_num_global_attentions, 
                                   aete_dimensionality=aete_dimensionality,
                                   encoder_dim=encoder_dim,
                                   dropout=dropout)
    if embedding_weight_path is not None:
        embedding_weights = torch.Tensor(np.load(embedding_weight_path)).cuda()
        decoder.load_pretrained_embeddings(embedding_weights)
        decoder.fine_tune_embeddings(fine_tune=True)
        
    decoder_optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, decoder.parameters()),
                                         lr=decoder_lr)
    encoder = Encoder(encoder_dim)
    encoder.fine_tune(fine_tune_encoder) # this refers only to the resnet part and does not refer to the transition layer 
    encoder_optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, encoder.parameters()),lr=encoder_lr) #if fine_tune_encoder else None
    start_epoch = 0
    epochs_since_improvement = 0
    best_auroc = 0
    decoder = decoder.cuda()
    encoder = encoder.cuda()
else:
    checkpoint = torch.load(checkpoint)
    start_epoch = checkpoint['epoch'] + 1
    epochs_since_improvement = checkpoint['epochs_since_improvement']
    best_auroc = checkpoint['overall_auroc']
    
    decoder = DecoderWithAttention(attention_dim=attention_dim,
                                   embed_dim=emb_dim,
                                   decoder_dim=decoder_dim,
                                   vocab_size=vocab_size,
                                   num_classes=num_classes, 
                                   aete_num_global_attentions=aete_num_global_attentions, 
                                   aete_dimensionality=aete_dimensionality,
                                   encoder_dim=encoder_dim,
                                   dropout=dropout)
    encoder = Encoder(encoder_dim)
    
#     decoder = checkpoint['decoder']
    decoder.load_state_dict(checkpoint['decoder'])
    encoder.load_state_dict(checkpoint['encoder'])
    
    decoder = decoder.cuda()
    encoder = encoder.cuda()
    
    decoder_optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, decoder.parameters()),
                                         lr=decoder_lr)
    encoder_optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, encoder.parameters()),
                                         lr=encoder_lr)
    decoder_optimizer.load_state_dict(checkpoint['decoder_optimizer'])
    encoder_optimizer.load_state_dict(checkpoint['encoder_optimizer'])
    if fine_tune_encoder is True and encoder_optimizer is None:
        encoder.fine_tune(fine_tune_encoder)
        encoder_optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, encoder.parameters()),
                                             lr=encoder_lr)




# Normalization and preprocessing the images 
normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # imageNet parameters     
transformList = []
transformList.append(transforms.RandomResizedCrop(transCrop))
transformList.append(transforms.RandomHorizontalFlip())
transformList.append(transforms.ToTensor())
transformList.append(normalize)      
transformSequence=transforms.Compose(transformList)


# Dataset loader 

# We might need to get the max sequence length in each dataset:
if longest_train is None:
    max_train = 0 
    for train_report_dir in train_report_dirs:
        all_reports_train = preprocess_all_reports(train_report_dir)
        longest_train = len(max(all_reports_train, key=len))+2 # add 2 for start/end tokens 
        if longest_train > max_train:
            max_train = longest_train 
        print("Longest report (train) in %s:"%train_report_dir, longest_train)
    longest_train = max_train 
if longest_val is None:
    all_reports_val = preprocess_all_reports(val_report_dir)
    longest_val = len(max(all_reports_val, key=len))+2 # add 2 for start/end tokens 
    print("Longest report (val): ", longest_val)
    
datasetTrain = TienetTrainDatasetGenerator(pathImageDirectory=pathDirDataTrain, 
                                           pathReportDirectory=pathDirDataTrain,
                                           pathDatasetFile=pathFileTrain,
                                           embeddingModelFile=embeddingModelFile,
                                           transform=transformSequence, 
                                           maxReportLength=longest_train)
train_loader = DataLoader(dataset=datasetTrain, batch_size=batch_size, shuffle=True,  
                             num_workers=num_workers, pin_memory=True)
if pathDirDataVal != None:
    datasetVal =   TienetTrainDatasetGenerator(pathImageDirectory=pathDirDataVal, 
                                               pathReportDirectory=pathDirDataVal, 
                                               pathDatasetFile=pathFileVal, 
                                               embeddingModelFile=embeddingModelFile,
                                               transform=transformSequence, 
                                               maxReportLength=longest_val)
    val_loader = DataLoader(dataset=datasetVal, batch_size=batch_size, shuffle=False, 
                               num_workers=num_workers, pin_memory=True)

    
# Loss, use masked_BCE
criterion = TieNetLoss(alpha=alpha, beta_P=beta_P, beta_N=beta_N, lambda_m=lambda_m)


# Epochs
for epoch in range(start_epoch, epochs):

    # Decay learning rate if there is no improvement for 8 consecutive epochs, and terminate training after 20
    if epochs_since_improvement == patience:
        break
    ## The below is only if we want to adjust the learning rate, but the paper just keeps a constant learning rate
#     if epochs_since_improvement > 0 and epochs_since_improvement % 8 == 0:
#         adjust_learning_rate(decoder_optimizer, 0.8)
#         if fine_tune_encoder:
#             adjust_learning_rate(encoder_optimizer, 0.8)

    # One epoch's training
    train(train_loader=train_loader,
          encoder=encoder,
          decoder=decoder,
          criterion=criterion,
          encoder_optimizer=encoder_optimizer,
          decoder_optimizer=decoder_optimizer,
          epoch=epoch)

    # One epoch's validation
    if pathDirDataVal != None:
        recent_aurocs  = validate(val_loader=val_loader,
                            encoder=encoder,
                            decoder=decoder,
                            criterion=criterion)
        recent_auroc = recent_aurocs[-1] # the last AUC value 
        class_aurocs = recent_aurocs[:-1]

        # Check if there was an improvement
        is_best = recent_auroc > best_auroc
        best_auroc = max(recent_auroc, best_auroc)
        if not is_best:
            epochs_since_improvement += 1
            print("\nEpochs since last improvement: %d\n" % (epochs_since_improvement,))
        else:
            epochs_since_improvement = 0
    else:
        is_best = False
        class_aurocs = 0 
        recent_auroc = 0 
        
    # Save checkpoint
    save_checkpoint(data_name, epoch, epochs_since_improvement, encoder, decoder, encoder_optimizer,
                    decoder_optimizer, recent_auroc, class_aurocs, is_best)


Epoch: [0][0/3331]	Batch Time 2.316 (2.316)	Data Load Time 1.269 (1.269)	Loss 7.4572 (7.4572)	
Epoch: [0][100/3331]	Batch Time 0.770 (0.894)	Data Load Time 0.002 (0.014)	Loss 3.1048 (4.7067)	
Epoch: [0][200/3331]	Batch Time 0.902 (0.889)	Data Load Time 0.002 (0.008)	Loss 2.5444 (3.7761)	
Epoch: [0][300/3331]	Batch Time 0.814 (0.868)	Data Load Time 0.002 (0.006)	Loss 2.5773 (3.3552)	
Epoch: [0][400/3331]	Batch Time 0.850 (0.865)	Data Load Time 0.002 (0.005)	Loss 2.3226 (3.1081)	
Epoch: [0][500/3331]	Batch Time 0.853 (0.860)	Data Load Time 0.001 (0.004)	Loss 2.2706 (2.9430)	
Epoch: [0][600/3331]	Batch Time 0.865 (0.853)	Data Load Time 0.001 (0.004)	Loss 2.2252 (2.8177)	
Epoch: [0][700/3331]	Batch Time 0.868 (0.844)	Data Load Time 0.001 (0.004)	Loss 2.2099 (2.7191)	
Epoch: [0][800/3331]	Batch Time 0.790 (0.843)	Data Load Time 0.002 (0.003)	Loss 2.0843 (2.6426)	
Epoch: [0][900/3331]	Batch Time 0.741 (0.839)	Data Load Time 0.001 (0.003)	Loss 1.8125 (2.5768)	
Epoch: [0][1000/3331]	Batch Time

[Jump back to beginning](#start)<br>

## Evaluation

<a id='eval'></a>

In [ ]:

pathModel= 'models/checkpoint_tienet20k_dualloss.pth.tar'
pathDirDataTest = '..'
pathFileTest = 'dataset/test_tienet.txt'
embeddingModelFile = 'data_4_pubmed_embeddings'


# Preprocessing
report_exists = True # if it does, then the pathFileTest txt file needs to be adjusted accordingly to extract label 
img_size = 224 # size of the final image
start_token_index = 7708 # the index for the start token 
end_token_index = 7709 
max_report_length = 300 # maximum number of words to be sampled for a given report 

# Model parameters
emb_dim = 200  # dimension of word embeddings
vocab_size = 7710 # number of words in vocabulary, including <start>, <end>, and pad index/unknown words (0)
encoder_dim = 1024 # dimension of the encoded images 
attention_dim = 350  # dimension of attention linear layers
decoder_dim = 256 # dimension of decoder RNN
aete_num_global_attentions = 5 # for attention encoded text embedding
aete_dimensionality = 2000 # for attention encoded text embedding
num_classes = 14 # number of classes for classification
dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # sets device for model and PyTorch tensors
cudnn.benchmark = False  # set to true only if inputs to model are fixed size; otherwise lot of computational overhead

k = 1 # the number of sequences to sample for each given report  



##############

encoder = Encoder(encoder_dim)
decoder = DecoderWithAttention(attention_dim=attention_dim,
                                   embed_dim=emb_dim,
                                   decoder_dim=decoder_dim,
                                   vocab_size=vocab_size,
                                   num_classes=num_classes, 
                                   aete_num_global_attentions=aete_num_global_attentions, 
                                   aete_dimensionality=aete_dimensionality,
                                   encoder_dim=encoder_dim,
                                   dropout=dropout)



checkpoint = torch.load(pathModel)
encoder.load_state_dict(checkpoint['encoder'])
decoder.load_state_dict(checkpoint['decoder'])
encoder = encoder.cuda()
decoder = decoder.cuda()
encoder.eval()
decoder.eval()



#-------------------- SETTINGS: DATASET BUILDERS
transformList = []
transformList.append(transforms.Resize(img_size))
transformList.append(transforms.ToTensor())
transformList.append(transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]))
transformSequence = transforms.Compose(transformList)

# DataLoader
datasetTest = DatasetGenerator(pathImageDirectory=pathDirDataTest, 
                                           pathDatasetFile=pathFileTest,
                                           transform=transformSequence,
                                          report_exists=True)
test_loader = DataLoader(dataset=datasetTest, batch_size=1, shuffle=False, 
                           num_workers=1, pin_memory=True)


# For each image
outputs = torch.FloatTensor().to(device)
labels = torch.FloatTensor().to(device)
with torch.no_grad():
    for idx, (image, label) in tqdm(enumerate(test_loader)):
        with torch.no_grad():
            # Move to GPU device, if available
            image = image.to(device)  # (1, 3, 224,224)

            # Encode
            encoder_out = encoder(image)  # (1, enc_image_size, enc_image_size, encoder_dim)
            enc_image_size = encoder_out.size(1)
            encoder_dim = encoder_out.size(3)

            # Flatten encoding
            encoder_out_img = encoder_out.view(1, -1, encoder_dim)  # (1, num_pixels, encoder_dim)
            num_pixels = encoder_out_img.size(1)

            # We'll treat the problem as having a batch size of k
            encoder_out = encoder_out_img.expand(k, num_pixels, encoder_dim)  # (k, num_pixels, encoder_dim)

            # Tensor to store top k previous words at each step; now they're just <start>
            k_prev_words = torch.LongTensor([start_token_index] * k).to(device)  # (k,)
            
            # Tensor to store top k sequences; now they're just <start>
            seqs = k_prev_words.unsqueeze(-1)  # (k, 1)

            # Tensor to store top k sequences' scores; now they're just 0
            top_k_scores = torch.zeros(k, 1).to(device)  # (k, 1)

            # Lists to store completed sequences and scores
            complete_seqs = list()
            complete_seqs_scores = list()

            # Start decoding
            step = 1
            h, c = decoder.init_hidden_state(encoder_out)

            complete_alphas = []
            complete_hiddens = [] 
            alphas = torch.FloatTensor().to(device)
            hiddens = torch.FloatTensor().to(device)
            # s is a number less than or equal to k, because sequences are removed from this process once they hit <end>
            while True:
                embeddings = decoder.embedding(k_prev_words).squeeze(1)  # (s, embed_dim)

                awe, alpha = decoder.attention(encoder_out, h)  # (s, encoder_dim) [attention_embedding], (s, num_pixels) [alphas]

                gate = decoder.sigmoid(decoder.f_beta(h))  # gating scalar, (s, encoder_dim)
                awe = gate * awe

                h, c = decoder.decode_step(torch.cat([embeddings, awe], dim=1), (h, c))  # (s, decoder_dim)


                #### Now, sample for next words 
                scores = decoder.fc(h)  # (s, vocab_size)
                scores = F.log_softmax(scores, dim=1)

                # Add
                scores = top_k_scores.expand_as(scores) + scores  # (s, vocab_size)

                # For the first step, all k points will have the same scores (since same k previous words, h, c)
                if step == 1:
                    top_k_scores, top_k_words = scores[0].topk(k, 0, True, True)  # (s)
                else:
                    # Unroll and find top scores, and their unrolled indices
                    top_k_scores, top_k_words = scores.view(-1).topk(k, 0, True, True)  # (s)

                # Convert unrolled indices to actual indices of scores
                prev_word_inds = top_k_words / vocab_size  # (s)
                next_word_inds = top_k_words % vocab_size  # (s)

                # Add new words to sequences
                seqs = torch.cat([seqs[prev_word_inds], next_word_inds.unsqueeze(1)], dim=1)  # (s, step+1)
                # Add new alphas and new hiddens to the sequences 
                hiddens = torch.cat([hiddens,h.unsqueeze(1)], dim=1) # h.squeeze(1) has dim (s,1,decoder_dim) 
                alphas = torch.cat([alphas,alpha.unsqueeze(1)],dim=1) # alpha.squeeze(1) has dim (s,1,num_pixels)

                # Which sequences are incomplete (didn't reach <end>)?
                incomplete_inds = [ind for ind, next_word in enumerate(next_word_inds) if
                                   next_word != end_token_index]
                complete_inds = list(set(range(len(next_word_inds))) - set(incomplete_inds))
                # Set aside complete sequences
                if len(complete_inds) > 0:
                    complete_seqs.extend(seqs[complete_inds].tolist())
                    complete_seqs_scores.extend(top_k_scores[complete_inds])
                    complete_hiddens.extend(hiddens[complete_inds].tolist())
                    complete_alphas.extend(alphas[complete_inds].tolist())
                    break
#                     k -= len(complete_inds)  
#                     k = 0 
                    
#                     if k == 0:
#                         print('shouldnt we be here')
#                         break

                # Proceed with incomplete sequences otherwise
                
    #             
                seqs = seqs[incomplete_inds]
                alphas = alphas[incomplete_inds]
                hiddens = hiddens[incomplete_inds]
                h = h[prev_word_inds[incomplete_inds]]
                c = c[prev_word_inds[incomplete_inds]]
                encoder_out = encoder_out[prev_word_inds[incomplete_inds]]
                top_k_scores = top_k_scores[incomplete_inds].unsqueeze(1)
                
                k_prev_words = next_word_inds[incomplete_inds].unsqueeze(1) #this causes kernel breakdown
#                 k_prev_words = next_word_inds
    
                # Break if things have been going on too long
                if step > max_report_length:
                    # Add all unfinished to the complete list 
                    complete_seqs.extend(seqs[incomplete_inds].tolist())
                    complete_seqs_scores.extend(top_k_scores[incomplete_inds])
                    complete_hiddens.extend(hiddens[incomplete_inds].tolist())
                    complete_alphas.extend(alphas[incomplete_inds].tolist())
                    break
                step += 1

            i = complete_seqs_scores.index(max(complete_seqs_scores))
            seq = complete_seqs[i]
            final_hidden = torch.Tensor(complete_hiddens[i]).squeeze(0).cuda()
            final_alpha = torch.Tensor(complete_alphas[i]).squeeze(0).cuda()
            #Should we write out a report just to see? 
            ## TODO: write out a report. 

            #Finish up with AETE and SW_GAP
            x_aete, g = decoder.aete(final_hidden.unsqueeze(0), [final_hidden.size(0)])
            x_swgap = decoder.swgap(g, final_alpha.unsqueeze(0), encoder_out_img)
            x = torch.cat([x_aete, x_swgap], dim=-1)
            output = decoder.fc_final(x)
            final_sigmoid = nn.Sigmoid()
            output = final_sigmoid(output)
            outputs = torch.cat([outputs,output], dim=0)
            labels = torch.cat([labels,label.cuda()], dim=0)
            
            if idx % 1000 == 0 and idx != 0:
                aucs = computeAUROC(labels, outputs, num_classes)
                print('At index %d, the overall AUC is:', aucs[-1])
aucs = computeAUROC(labels, outputs, num_classes)
print('Overall Test AUC:',aucs[-1],'\nOther AUCs:',aucs[-1])

At index %d, the overall AUC is: 0.5758053285875293
At index %d, the overall AUC is: 0.5712111498891302
At index %d, the overall AUC is: 0.5669990281439589
At index %d, the overall AUC is: 0.5700232250227784
At index %d, the overall AUC is: 0.5736844809961921


In [12]:
model = KeyedVectors.load(embeddingModelFile)

In [13]:
report = [model.index2word[ind-1] for ind in seq[1:-1]]

In [14]:
report

['final',
 'report',
 'examination',
 'chest',
 'pa',
 'and',
 'lat',
 'indication',
 'year',
 'old',
 'woman',
 'with',
 'cough',
 'and',
 'fever',
 'ro',
 'pna',
 'ro',
 'pna',
 'impression',
 'in',
 'comparison',
 'with',
 'the',
 'study',
 'of',
 'there',
 'is',
 'little',
 'change',
 'and',
 'no',
 'evidence',
 'of',
 'acute',
 'cardiopulmonary',
 'disease',
 'no',
 'pneumonia',
 'vascular',
 'congestion',
 'or',
 'pleural',
 'effusion']